# Nextflow

[Nextflow](https://www.nextflow.io/) is a workflow management system used for executing scientific workflows across platforms scalably, portably, and reproducibly.

Here, we'll run a demo of the microscopy pipeline [mcmicro](https://github.com/labsyspharm/mcmicro) to correct uneven illumination. [Reference](https://mcmicro.org/)

```{note}

Typically, you run the Nextflow workflow from the command line or Seqera Platform and then register input and output data with a script.
The Seqera Platform allows for [post-run scripts](https://docs.seqera.io/platform/23.4.0/launch/advanced#pre-and-post-run-scripts) that can automate this process.
```

In [ ]:
!lamin init --storage . --name nextflow-mcmicro

In [ ]:
import lamindb as ln

## Run mcmicro/exemplar pipeline

Register the `exemplar-001` data as input of the mcmicro pipeline by running the script below:

In [ ]:
from IPython.display import display, Code

with open('mcmicro_exemplar.py', 'r') as file:
    display(Code(file.read(), language='python'))

In [ ]:
!python mcmicro_exemplar.py exemplar-001

## Run mcmicro pipeline

Run mcmicro pipeline and track input/output data with the script below:

In [ ]:
with open('mcmicro.py', 'r') as file:
    display(Code(file.read(), language='python'))

In [ ]:
!python mcmicro.py exemplar-001

## Data lineage

View data lineage:

In [ ]:
output = ln.Artifact.filter(key__icontains="exemplar-001.ome.tif").one()

In [ ]:
output.view_lineage()

View the database content:

In [ ]:
ln.view()

In [ ]:
# clean up the test instance:
!lamin delete --force nextflow-mcmicro